In [0]:
landing_path='s3://databricks-practice-ro/landing'
checkpoint_path='s3://databricks-practice-ro/checkpoints/bronze_data'
schema_path="s3://databricks-practice-ro/checkpoints/bronze_schema"
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
nyc_schema = StructType([
    StructField("vendor_id", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("rate_code_id", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", StringType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])



In [0]:
df_bronze=(spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','json')\
        .option('cloudFiles.rescuedDataColumn','_rescued_data')\
            .option('cloudFiles.schemaLocation',schema_path)
            .schema(nyc_schema)\
                .load(landing_path)
        )

query=(df_bronze.writeStream.format('delta')\
    .option('mergeSchema','true')\
        .option('checkpointLocation',checkpoint_path)\
            .outputMode('Append')\
                .trigger(availableNow=True)
            .table('tester.default.bronze_data'))

query.awaitTermination()

count = spark.read.table('tester.default.bronze_data').count()
print(f"✅ Ingestion Complete. Total rows in Bronze: {count}")